In [1]:
import bs4 as bs
import urllib.request
import lxml
import requests
import pandas as pd

In [2]:
# defining a general function to save the dataframe as CSV file
def save_csv(df, name):
    if name[-4:] == ".csv":
        df.to_csv(name)
    else:
        name = name + ".csv"
        df.to_csv(name)
    print("your file",name, "has been saved")
    df.head()
    #print(df.head())

**1. fetching the headertags from Wikipedia mainpage**

In [17]:
def webscrap_wiki(link, csv_name):
    # fetching the wbpage
    print("fetching the data from the webpage...")
    page = requests.get("https://en.wikipedia.org/wiki/Main_Page")
    soup = bs.BeautifulSoup(page.text, "html.parser")
    
    # extracting the h1 tags
    h2_tags = soup.select("h2.mp-h2 span")
    h2_tags = [tag.text for tag in h2_tags]
    h2_tags = [tags for tags in h2_tags if len(tags)>0]
    
    # extracting the h2 tags
    h3_tags = soup.select("h3.vector-menu-heading span")
    h3_tags = [h3tag.text for h3tag in h3_tags]
    h3_tags = [htags for htags in h3_tags if len(htags)>0]
    
    # concatenate h2 and h3 list
    final_tags = h2_tags + h3_tags
    
    # creating a dictionary of all the list created above
    dict = {'header tags': final_tags}
    df = pd.DataFrame(dict)
    return df.head()
    
    # df can be saved as csv file by executing the function below
    #save_csv(df, "wikipedia headertags")

webscrap_wiki("https://en.wikipedia.org/wiki/Main_Page", "Wikipedia headertags")

fetching the data from the webpage...


,header tags
0,From today's featured article
1,Did you know ...
2,In the news
3,On this day
4,From today's featured list


**2. IMDB's top rated 100 movies**

In [19]:
# defining function to web scrape the web page
# paste the link in the webscrap function
def webscrap_IMDB(link, csv_name):
    # fetching the wbpage
    print("fetching the data from the webpage...")
    page = requests.get(link)
    soup = bs.BeautifulSoup(page.text, "html.parser")
    # extracting the movietags and innertags
    movietags = soup.select("td.titleColumn")
    inner_movietags = soup.select("td.titleColumn a")
    # extracting the name of the movie
    print("fetching movie names...")
    movie_names = [movie.text for movie in inner_movietags]
    # extracting the actors including the director
    # year of release of movie
    print("fetching year of release...")
    years = [movie.text.split()[-1].replace(")","").replace("(","") for movie in movietags]
    # imdb rating tag of the movie 
    rating_tags = soup.select("td.posterColumn span[name = ir]")
    # imdb rating of movie
    print("fetching movie rating...")
    rating = [round(float(movie['data-value']),2) for movie in rating_tags]
    # creating a dictionary of all the list created above
    dict = {'movie name': movie_names, "year": years,'rating': rating} 
    # converting the dictionary into dataframe
    df = pd.DataFrame(dict)
    df["year"] = df["year"].apply(pd.to_numeric)
    df_100 = df.head(100)
    print("creating dataframe...")
    print("saving dataframe as CSV...")
    save_csv(df_100, csv_name)
    return df_100.head()
    #return (df_100)
webscrap_IMDB("https://www.imdb.com/chart/top/?ref_=nv_mv_250/", "top_100_movies")

fetching the data from the webpage...
fetching movie names...
fetching year of release...
fetching movie rating...
creating dataframe...
saving dataframe as CSV...
your file top_100_movies.csv has been saved


,movie name,year,rating
0,The Shawshank Redemption,1994,9.22
1,The Godfather,1972,9.15
2,The Godfather: Part II,1974,8.98
3,The Dark Knight,2008,8.97
4,12 Angry Men,1957,8.94


**3. IMDB's top 100 indian movies**
Same algorithm used in above problem can be used to scrap the top 100 indian movies. Just replace the link of top 100 movies with the link of top indian movies.

In [20]:
webscrap_IMDB('''https://www.imdb.com/india/top-rated-indian-movies/?pf_rd_m=A2FGELUUNOQJNL&pf_
rd_p=2e9dfa9b-3e4d-4d39-acd2-8af11f252a59&pf_rd_r=RFJGXDAT1JGH01JGF9CY&pf_rd_s=right-5&pf_
rd_t=15506&pf_rd_i=top&ref_=chttp_india_tr_rhs_1''', "top_100_indian_movies")

fetching the data from the webpage...
fetching movie names...
fetching year of release...
fetching movie rating...
creating dataframe...
saving dataframe as CSV...
your file top_100_indian_movies.csv has been saved


,movie name,year,rating
0,Nayakan,1987,8.49
1,Anbe Sivam,2003,8.48
2,Pariyerum Perumal,2018,8.48
3,C/o Kancharapalem,2018,8.47
4,Golmaal,1979,8.47


**4. getting bookname, author, genre, and review**

In [15]:
bookslink = "https://bookpage.com/reviews"

def book_rev(link, csv_name):
    print("fetching page...")
    page = requests.get(bookslink)
    soup = bs.BeautifulSoup(page.text, "html.parser")

    # extracting book names
    print("getting book names...")
    book_names = soup.select("h4")
    book_names = [hdtag.text.strip().replace("★","") for hdtag in book_names]
    
    print("getting book author...")
    author = soup.select("p.sans.bold")
    author = [author.text.strip() for author in author]
    
    print("getting book genre...")
    genre = soup.select("p.genre-links.hidden-phone")
    genre = [genre.text.strip().replace("\n","") for genre in genre]
    
    def get_rev(rev_link):
        rl_page = requests.get(rev_link)
        soup_rl = bs.BeautifulSoup(rl_page.text, "html.parser")
        review = soup_rl.select("div.span7 p")
        review = [review.text.replace("\n",'') for review in review]
        r=""
        for rev in review:
            r = r + rev
        return r
    
    print("getting review...")
    review_link = soup.select("div.read-full a")
    review_link = ['http://bookpage.com'+rev.get('href') for rev in review_link]
    reviews = [get_rev(rev) for rev in review_link]
    
    #creating dictionary and dataframe
    dict = {"book name": book_names, "author": author, "genre": genre, "reviews": reviews}
    review_df = pd.DataFrame(dict)
    df = review_df.head()
    return df
    save_csv(review_df, csv_name)
book_rev(bookslink, "books_review")

fetching page...
getting book names...
getting book author...
getting book genre...
getting review...


,book name,author,genre,reviews
0,The Lincoln Highway,Amor Towles,Fiction / Historical Fiction,"“I guess you haven’t had your adventure yet,” ..."
1,Light From Uncommon Stars,Ryka Aoki,Science Fiction & Fantasy / Science Fiction,What is it with the devil and violinists? Seem...
2,Chasing Ghosts,Marc Hartzman,Nonfiction / History / Spirituality,"As long as there has been death, there have be..."
3,Mr. Watson's Chickens,"Jarrett Dapier, Andrea Tsurumi",Children's / Picture Book,Mr. Watson and Mr. Nelson live together in a “...
4,A Calling for Charlie Barnes,Joshua Ferris,Fiction / Family Drama,How does one sum up the arc of a long life? Th...


**5(i). scraping top 10 ODI terms with matches, points and ratings**

In [14]:
odi_link = "https://www.icc-cricket.com/rankings/mens/team-rankings/odi"

def top_odi(link, csv_name):
    print("getting details of teams...")
    #getting details of top team
    
    page = requests.get(link)
    soup = bs.BeautifulSoup(page.text, "html.parser")
    team = soup.select("span.u-hide-phablet")
    team = [team.text for team in team if len(team.text)>0]
    matches = [soup.select("td.rankings-block__banner--matches")[0].text]
    points = [soup.select("td.rankings-block__banner--points")[0].text]
    rating = [soup.select("td.rankings-block__banner--rating.u-text-right")[0].text.strip()]

    #getting details of remaining teams
    ratings = soup.select("td.table-body__cell.u-text-right.rating")
    ratings = [rating.text for rating in ratings]
    ratings = rating + ratings

    matches_points = soup.select("td.table-body__cell.u-center-text")
    for i in range(len(matches_points)):
        if i%2==0:
            matches.append(matches_points[i].text)
        else:
            points.append(matches_points[i].text)
    
    dict = {"team":team[0:10], "matches":matches[0:10], "points":points[0:10], "ratings": ratings[0:10]}
    odi_df = pd.DataFrame(dict)
    df = odi_df.head()
    return df
    save_csv(odi_df, csv_name)
    
top_odi(odi_link, "top_odi")

getting details of teams...


,team,matches,points,ratings
0,New Zealand,17,"2,054",121
1,England,32,"3,793",119
2,Australia,28,"3,244",116
3,India,32,"3,624",113
4,South Africa,25,"2,459",98


**5(ii). top 10 ODI batsman with their team and ratings**

In [13]:
odi_link_bat= "https://www.icc-cricket.com/rankings/mens/player-rankings/odi"

def top_odi_bat(link, csv_name):
    print("getting details of top batsman...")
    page = requests.get(link)
    soup = bs.BeautifulSoup(page.text, "html.parser")

    #extracting top most batsman
    top1 = soup.select("div.rankings-block__banner--name")
    #extracting the remaining nine batsman
    top = soup.select("td.table-body__cell.name a")[0:9]
    top_bats = [top1[0].text] + [top.text for top in top]

    top1 = soup.select("div.rankings-block__banner--name")
    top = soup.select("td.table-body__cell.name a")[0:9]
    top_bats = [top1[0].text] + [top.text for top in top]

    top_nat1 = soup.select('div.rankings-block__banner--nationality')[0].text.strip()
    top_nat = soup.select("td.table-body__cell.nationality-logo span.table-body__logo-text")[0:9]
    top_nats = [top_nat1] + [top.text for top in top_nat]

    top_rat1 = soup.select('div.rankings-block__banner--rating')[0].text.strip()
    top_rat = soup.select("td.table-body__cell.u-text-right.rating")[0:9]
    top_rats = [top_rat1] + [rat.text for rat in top_rat]
    
    dict = {"top batsmen":top_bats, "nationality": top_nats, "ratings":top_rats}
    bat_df = pd.DataFrame(dict)
    save_csv(bat_df, csv_name)
    df = bat_df.head()
    return df
top_odi_bat(odi_link_bat, "top_odi_bat")

getting details of top batsman...
your file top_odi_bat.csv has been saved


,top batsmen,nationality,ratings
0,Babar Azam,PAK,873
1,Virat Kohli,IND,844
2,Rohit Sharma,IND,813
3,Ross Taylor,NZ,801
4,Aaron Finch,AUS,779


**5(iii). scraping top 10 ODI bowlers with their team and ratings**

In [55]:
# in this i haven't defined functions for scraping and didn't saved data as CSV file
odi_top_bat = "https://www.icc-cricket.com/rankings/mens/player-rankings/odi"
page = requests.get(odi_top_bat)
soup = bs.BeautifulSoup(page.text, "html.parser")

top_bowl1 = soup.select("div.rankings-block__banner--name")
top_bowl = soup.select("td.table-body__cell.name a")[9:18]
top_bowls = [top_bowl1[1].text] + [top.text for top in top_bowl]

top_bowl_nat1 = soup.select('div.rankings-block__banner--nationality')[1].text.replace("\n",'')
top_bowl_nat = soup.select("td.table-body__cell.nationality-logo span.table-body__logo-text")[9:18]
top_bowl_nats = [top_bowl_nat1] + [top.text for top in top_bowl_nat]

top_bowl_rat1 = soup.select('div.rankings-block__banner--rating')[1].text.replace("\n",'')
top_bowl_rat = soup.select("td.table-body__cell.u-text-right.rating")[9:18]
top_bowl_rats = [top_bowl_rat1] + [top.text for top in top_bowl_rat]
    
dict = {"top bowler":top_bowls, "nationality": top_bowl_nats, "ratings":top_bowl_rats}
bowl_df = pd.DataFrame(dict)
bowl_df.head(5)

,top bowler,nationality,ratings
0,Trent Boult,NZ,737
1,Josh Hazlewood,AUS,709
2,Mujeeb Ur Rahman,AFG,708
3,Chris Woakes,ENG,700
4,Mehedi Hasan,BAN,692


**6(i) Fetching top ODI women team. The same code can work to fetching women ODI teams.**

In [21]:
# the code written for ODI men team can work for ODI women team also. Just replace the link of ODI women teams page
top_odi('https://www.icc-cricket.com/rankings/womens/team-rankings/odi', "top_odi_women")

getting details of teams...


,team,matches,points,ratings
0,Australia,21,"3,379",161
1,England,25,"2,983",119
2,South Africa,29,"3,390",117
3,India,26,"2,934",113
4,New Zealand,26,"2,392",92


**6(ii) fetching top women batsman for ODI**

In [6]:
women_bat_link = "https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting"
def w_top_bat(url, csv_name):
    print("fetching the player details...")
    page = requests.get(url)
    soup = bs.BeautifulSoup(page.text, "lxml")

    # fetching the details of first player
    top_1_bat = soup.select("div.rankings-block__banner--name-large")[0].text
    top_1_wbat_nat = soup.select('div.rankings-block__banner--nationality')[0].text.strip()
    top_1_wbat_rat = soup.select("div.rankings-block__banner--rating")[0].text.strip()

    # fetching the details of remaining batsman
    top_wbat = soup.select("td.table-body__cell.rankings-table__name.name") 
    top_wbat = [top.text.strip() for top in top_wbat]
    top_wnat = soup.select("td.table-body__cell.nationality-logo.rankings-table__team")
    top_wnat = [top.text.strip() for top in top_wnat]
    top_wrat = soup.select("td.table-body__cell.rating")
    top_wrat = [top.text.strip() for top in top_wrat]

    dict = {"top woman batsman":[top_1_bat]+top_wbat, "nationality": [top_1_wbat_nat]+top_wnat, "ratings":[top_1_wbat_rat]+top_wrat}
    wbat_df = pd.DataFrame(dict)
    return wbat_df.head(5)
w_top_bat(women_bat_link, "women_top_bat")

fetching the player details...


,top woman batsman,nationality,ratings
0,Lizelle Lee,SA,761
1,Alyssa Healy,AUS,750
2,Mithali Raj,IND,738
3,Tammy Beaumont,ENG,728
4,Amy Satterthwaite,NZ,717


**6(iii) Fetching the top women bowler in ODI**

The function defined to fetch the top women batsman can fetch the top women bowler with the apprpriate link.

In [8]:
w_top_bowl = w_top_bat

In [9]:
bowl_url = 'https://www.icc-cricket.com/rankings/womens/player-rankings/odi/bowling'
w_top_bowl(bowl_url, "women_top_bowler")

fetching the player details...


,top woman batsman,nationality,ratings
0,Jess Jonassen,AUS,760
1,Jhulan Goswami,IND,727
2,Megan Schutt,AUS,717
3,Marizanne Kapp,SA,715
4,Sophie Ecclestone,ENG,701


In [24]:
page = requests.get(bowl_url)
soup = bs.BeautifulSoup(page.text, "lxml")

# fetching the details of first player
top_1_bat = soup.select("div.rankings-block__banner--name-large")[0].text
top_1_wbat_nat = soup.select('div.rankings-block__banner--nationality')[0].text.strip()
top_1_wbat_rat = soup.select("div.rankings-block__banner--rating")[0].text.strip()

# fetching the details of remaining batsman
top_wbat = soup.select("td.table-body__cell.rankings-table__name.name") 
top_wbat = [top.text.strip() for top in top_wbat]
top_wnat = soup.select("td.table-body__cell.nationality-logo.rankings-table__team")
top_wnat = [top.text.strip() for top in top_wnat]
top_wrat = soup.select("td.table-body__cell.rating")
top_wrat = [top.text.strip() for top in top_wrat]

dict = {"top woman batsman":[top_1_bat]+top_wbat, "nationality": [top_1_wbat_nat]+top_wnat, "ratings":[top_1_wbat_rat]+top_wrat}
wbat_df = pd.DataFrame(dict)
wbat_df.head(5)

,top woman batsman,nationality,ratings
0,Jess Jonassen,AUS,760
1,Jhulan Goswami,IND,727
2,Megan Schutt,AUS,717
3,Marizanne Kapp,SA,715
4,Sophie Ecclestone,ENG,701


 **7 Fetching the mobile phones below Rs. 20000 from amazon**

In [67]:
# collecting links for all products for a given page number
def get_links(link):
    product_links = []
    img_links = []
    
    page_link = link
    page = requests.get(page_link)
    soup = bs.BeautifulSoup(page.text, "lxml")
    #print(len(soup))
    links_a = soup.select("h2 a")
    image_link = soup.select('img.s-image')
    img_links = img_links+[link.get("src") for link in image_link]
    product_links = product_links+['https://amazon.in'+link.get('href') for link in links_a]
    #page_link = "https://amazon.in" + soup.select("ul.a-pagination li.a-normal a")[0].get("href")
    return img_links, product_links

In [68]:
# getting the title, price and rating of products
def title_rating_price(soup):
    #price = float(soup.find('span',id="priceblock_ourprice").text.replace("₹","").replace(",",""))
    price = float(soup.select("td.a-span12 span")[0].text.replace("₹","").replace(",",""))
    if price<20000:
        title = soup.select("h1 span")[0].text.replace("\n","").split("(")[0]
        rating = float(soup.select("span.a-icon-alt")[0].text[:3])
        price = float(soup.select("td.a-span12 span")[0].text.replace("₹","").replace(",",""))
        #price = float(soup.find('span',id="priceblock_ourprice").text.replace("₹","").replace(",",""))
        form_factor = ab = soup.select("td.a-span9 span")[3].text
        return title, rating, price, form_factor
    return 0,0,0,0

In [69]:
# fetching the data and creating the dataframe
HEADERS ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}
def amazon(n_page):
    titles = []
    img_url = []
    prices = []
    ratings = []
    form_fact = []
    page_link2 = 'https://www.amazon.in/s?i=electronics&rh=n%3A1389401031&fs=true&page=2&qid=1632398261&ref=sr_pg_1'
    for i in range(n_page):
        page1 = requests.get(page_link2, headers = HEADERS)
        soup = bs.BeautifulSoup(page1.text, "html.parser")
        image_link, product_link = get_links(page_link2)
        while len(image_link)==0:
            image_link, product_link = get_links(page_link2)
        while len(soup.select("ul.a-pagination li.a-last a")) == 0:
            page_link2= "https://amazon.in"+soup.select("ul.a-pagination li.a-last a")[0].get("href")
        print("images", len(image_link),"fetched")
        for j in range(len(image_link)):
            #print("fetching the data for",i,j)
            #print(product_link[j])
            page = requests.get(product_link[j], headers = HEADERS)
            soup = bs.BeautifulSoup(page.content, "html")
            title, rating, price, form_factor = title_rating_price(soup)
            titles.append(title)
            img_url.append(image_link[j])
            prices.append(price)
            ratings.append(rating)
            form_fact.append(form_factor)

    dict = {"title": titles,"image link": img_url, "price":prices, "form factor":form_fact}
    df = pd.DataFrame(dict)
    df = df[df["form factor"]=="Smartphone"]
    return df.head()

In [70]:
amazon(4)

images 24 fetched
images 24 fetched
images 24 fetched
images 24 fetched


,title,image link,price,form factor
2,Redmi 9,https://m.media-amazon.com/images/I/71A9Vo1Bat...,10999.0,Smartphone
15,Redmi Note 10 Pro,https://m.media-amazon.com/images/I/81aQWPoGdO...,19999.0,Smartphone
26,Redmi 9,https://m.media-amazon.com/images/I/71A9Vo1Bat...,10999.0,Smartphone
39,Redmi Note 10 Pro,https://m.media-amazon.com/images/I/81aQWPoGdO...,19999.0,Smartphone
50,Redmi 9,https://m.media-amazon.com/images/I/71A9Vo1Bat...,10999.0,Smartphone


In [57]:
page_link = 'https://www.amazon.in/s?i=electronics&rh=n%3A1389401031&fs=true&page=2&qid=1632398261&ref=sr_pg_1'

**8 Fetching weather data**

In [87]:
weather_link = "https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.YVNq_bgza00"

def fetch_weather(weather_link):
    page = requests.get(weather_link)
    soup = bs.BeautifulSoup(page.text, "html.parser")
    p_name = soup.select("li.forecast-tombstone p.period-name")
    period_names = [pn.text for pn in p_name]
    short_dec = soup.select("li p.short-desc")
    short_decs = [sd.text for sd in short_dec]
    desc = soup.select("p img")
    desc = [d.get("alt").split(":")[1] for d in desc]
    temp = soup.select('li p.temp')
    temps = [temp.text for temp in temp]
    dict = {"period name":period_names, "short dec":short_decs, "temp":temps, "decription": desc}
    df = pd.DataFrame(dict)
    return df.head()

fetch_weather(weather_link)

,period name,short dec,temp,decription
0,Overnight,Mostly Clear,Low: 59 °F,"Mostly clear, with a low around 59. Southwest..."
1,Sunday,Sunny,High: 79 °F,"Sunny, with a high near 79. Light west southw..."
2,SundayNight,Partly Cloudy,Low: 57 °F,"Partly cloudy, with a low around 57. West sou..."
3,Monday,Mostly Sunny,High: 75 °F,"Mostly sunny, with a high near 75. West wind ..."
4,MondayNight,Partly Cloudy,Low: 56 °F,"Partly cloudy, with a low around 56. West sou..."


**9 Fetching data from internshala**

In [91]:
internshala_link = "https://internshala.com/fresher-jobs"
def jobs_data(internshala_link):
    page1 = requests.get(internshala_link)
    soup1 = bs.BeautifulSoup(page1.text, "html.parser")
    job_title = soup1.select("div.company div.heading_4_5.profile a")
    job_title = [job.text.strip() for job in job_title]
    company = soup1.select("div.company div.heading_6.company_name a")
    company = [company.text.strip() for company in company]
    r = soup1.select("div.internship_other_details_container div.other_detail_item div.item_body")
    list = [ra.text.strip() for ra in r]
    CTC = [list[i+1] for i in range(len(list)) if i%3==0 ]
    apply_date = [list[i+2] for i in range(len(list)) if i%3==0 ]
    dict = {"job title":job_title, "company": company, "CTC": CTC, "apply date": apply_date}
    df = pd.DataFrame(dict)
    return df.head()

jobs_data(internshala_link)

,job title,company,CTC,apply date
0,Business Development Executive,NSE Academy,3 LPA,31 Oct' 21
1,Management Trainee - Sales,CEAT Limited,5 LPA,29 Oct' 21
2,Business Development Manager (Digital Marketin...,Graygraph Technologies LLC,4.2 - 4.8 LPA,1 Nov' 21
3,Corporate Sales Executive,Samvedna Senior Care Private Limited,3.25 - 4 LPA,1 Nov' 21
4,Junior Content Writer,High School Moms,3 - 3.2 LPA,1 Nov' 21


**10 fetching data from nobroker.com for a location**

In [54]:
#HEADERS ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
#(KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

banglore_url = 'https://www.nobroker.in/property/sale/bangalore/Electronic%20City?type=BHK4&searchParam=W3sibGF0IjoxMi44NDUyMTQ1LCJsb24iOjc3LjY2MDE2OTUsInBsYWNlSWQiOiJDaElKdy1GUWQ0cHNyanNSSGZkYXpnXzhYRW8iLCJwbGFjZU5hbWUiOiJFbGVjdHJvbmljIENpdHkifV0=&propertyAge=0&radius=2.0'
delhi_url = 'https://www.nobroker.in/property/sale/delhi/Sarita%20Vihar?searchParam=W3sibGF0IjoyOC41MzA3ODEyLCJsb24iOjc3LjI5Mzk5ODEsInBsYWNlSWQiOiJDaElKdTNJNlJxbm1ERGtSQUVraFFCY3QxU1UiLCJwbGFjZU5hbWUiOiJTYXJpdGEgVmloYXIifV0=&radius=2.0'
mumbai_url = 'https://www.nobroker.in/property/sale/mumbai/Film%20City%20Complex?searchParam=W3sibGF0IjoxOS4xNTk0MDM5LCJsb24iOjcyLjg4ODQ2OTgsInBsYWNlSWQiOiJDaElKcnhmT3lZMjM1enNSVkhSTG1vc1d2bHciLCJwbGFjZU5hbWUiOiJGaWxtIENpdHkgQ29tcGxleCJ9XQ==&radius=2.0'
def no_broker(link, csv_name):
    print("ferching data from nobroker")
    page = requests.get(link)
    soup = bs.BeautifulSoup(page.content, "lxml")
    print("fetching house details")
    h_list = soup.find_all('h2', class_='heading-6 font-semi-bold nb__1AShY')
    h_list = [house.text.strip() for house in h_list]
    location = soup.find_all('div',class_="nb__2CMjv")
    location = [loc.text.strip() for loc in location]
    detail = soup.find_all('div',class_="font-semi-bold heading-6")
    detail = [d.text for d in detail]
    area = [detail[i] for i in range(len(detail)) if i%3 == 0]
    emi = [detail[i+1] for i in range(len(detail)) if i%3 == 0]
    price = [detail[i+2] for i in range(len(detail)) if i%3 == 0]
    print("creating data frame")
    dict = {"title":h_list, "location": location, "area":area, "EMI":emi, "price":price}
    df = pd.DataFrame(dict)
    save_csv(df, csv_name)
    return df

In [56]:
no_broker(mumbai_url, "xyz")

ferching data from nobroker
fetching house details
creating data frame
your file xyz.csv has been saved


,title,location,area,EMI,price
0,1 RK In Independent House For Sale In Goregaon,"Independent House, Royal Palms, Aarey Colony,N...",150 sqft,"₹6,877/Month",₹12 Lacs


In [58]:
no_broker(banglore_url, "no_broker1")

ferching data from nobroker
fetching house details
creating data frame
your file no_broker1.csv has been saved


,title,location,area,EMI,price
0,4 BHK In Independent House For Sale In Hebba...,"Independent House, Bangalore - Hosur Road, Nea...","1,800 sqft","₹85,971/Month",₹1.5 Crores
1,4 BHK In Independent House For Sale In Elect...,"Independent House, surya nagar face 1","3,000 sqft",₹1.43 Lacs/Month,₹2.5 Crores
2,4 BHK In Independent House For Sale In Elect...,"Independent House, Hosur Rd,Near Infosys Limited","1,200 sqft","₹42,985/Month",₹75 Lacs
3,4 BHK In Independent House For Sale In Anant...,"Independent House, Glass factory Outlet nd cro...","2,200 sqft","₹45,851/Month",₹80 Lacs
4,4 BHK Flat For Sale In Sobha Silicon Oasis ...,"Sobha Silicon Oasis Naganathapura, Rayasandra ...","1,879 sqft","₹88,837/Month",₹1.55 Crores
5,4 BHK Apartment For Sale In Gopalan Gardenia...,"Gopalan Gardenia Gopalan gardenia, Veerasandr...","2,650 sqft","₹68,777/Month",₹1.2 Crores
6,4 BHK For Sale In Gpr Royale In Gpr Royale,6th Cross,"3,100 sqft","₹85,971/Month",₹1.5 Crores
7,4 BHK Flat For Sale In Electronic City,"Standalone Building, Shikaripalya near Shams S...","1,400 sqft","₹20,060/Month",₹35 Lacs
8,4 BHK In Independent House For Sale In Sarja...,"Independent House, Shantipura Village , S.P L...","1,100 sqft","₹40,120/Month",₹70 Lacs
9,4 BHK In Independent House For Sale In Nagan...,"Independent House, Doddanagamangala Rd opposit...","1,500 sqft","₹45,851/Month",₹80 Lacs
